In [1]:
using PauliPropagation

In [ ]:
function build_dqa_tfim_circuit_and_params(nq; P=64, tau=P, J=1.0, Γ=1.0, ħ=1.0, periodic=false, topology=nothing)
    del_t = tau / P # tau is total annealing time, divided into P steps
    # 1) Circuit: brickwork RX then RZZ per layer (corresponds to TFIM trotter circuit)
    if topology === nothing
        topology = bricklayertopology(nq; periodic=periodic)
    end
    circuit = tfitrottercircuit(nq, P; topology=topology)   # P “layers” (X-block then ZZ-block each)

    # 2) Parameter vector in Schrödinger order (the library reverses internally for Heisenberg)
    # Per layer m: first nq X-angles (one per qubit), then nb ZZ-angles (one per bond).
    nbonds  = periodic ? nq : (nq - 1) # if periodic nq bonds, else nq-1 (1d chain )
    total   = P * (nq + nbonds)
    params  = Vector{Float64}(undef, total)

    idx = 1
    for m in 1:P # steps of annealing
        s_m   = m / P
        gamma_m   = s_m * del_t / ħ           # paper Eq. (12)
        beta_m   = del_t / ħ                 # for TFIM: constant
        theta_X    = 2*beta_m * G               # angle for e^{-i theta_X X_j/2} - PauliRotation implementation in PP #constant for TFIM!
        theta_ZZ   = 2*gamma_m * J                # angle for e^{-i theta_ZZ Z_j Z_{j+1}/2}

        @inbounds fill!(view(params, idx:idx+nq-1), theta_X);             idx += nq
        @inbounds fill!(view(params, idx:idx+nbonds-1), theta_ZZ);       idx += nbonds
    end

    return circuit, params
end

# --- demo ---
nq = 4; P = 1000; tau = 1.0; J = 1.0; G = 1.0
circuit, params = build_dqa_tfim_circuit_and_params(nq; P=P, tau=tau, J=J, Γ=Γ, periodic=false)
#println(circuit)
println(params)
topology = bricklayertopology(nq; periodic=false)
# choose an observable, truncations, then propagate
H_tfim = PauliSum(nq)

for qind in 1:nq
    add!(H_tfim, :X, qind,Γ)
end

for pair in topology
    add!(H_tfim, [:Z, :Z], collect(pair), J)
end

max_weight = 10
min_abs_coeff = 1e-7
propagated = propagate(circuit, H_tfim, params; max_weight, min_abs_coeff)


[0.002, 0.002, 0.002, 0.002, 2.0e-6, 2.0e-6, 2.0e-6, 0.002, 0.002, 0.002, 0.002, 4.0e-6, 4.0e-6, 4.0e-6, 0.002, 0.002, 0.002, 0.002, 6.0e-6, 6.0e-6, 6.0e-6, 0.002, 0.002, 0.002, 0.002, 8.0e-6, 8.0e-6, 8.0e-6, 0.002, 0.002, 0.002, 0.002, 1.0e-5, 1.0e-5, 1.0e-5, 0.002, 0.002, 0.002, 0.002, 1.2e-5, 1.2e-5, 1.2e-5, 0.002, 0.002, 0.002, 0.002, 1.4e-5, 1.4e-5, 1.4e-5, 0.002, 0.002, 0.002, 0.002, 1.6e-5, 1.6e-5, 1.6e-5, 0.002, 0.002, 0.002, 0.002, 1.8e-5, 1.8e-5, 1.8e-5, 0.002, 0.002, 0.002, 0.002, 2.0e-5, 2.0e-5, 2.0e-5, 0.002, 0.002, 0.002, 0.002, 2.2e-5, 2.2e-5, 2.2e-5, 0.002, 0.002, 0.002, 0.002, 2.4e-5, 2.4e-5, 2.4e-5, 0.002, 0.002, 0.002, 0.002, 2.6e-5, 2.6e-5, 2.6e-5, 0.002, 0.002, 0.002, 0.002, 2.8e-5, 2.8e-5, 2.8e-5, 0.002, 0.002, 0.002, 0.002, 3.0e-5, 3.0e-5, 3.0e-5, 0.002, 0.002, 0.002, 0.002, 3.2e-5, 3.2e-5, 3.2e-5, 0.002, 0.002, 0.002, 0.002, 3.4000000000000007e-5, 3.4000000000000007e-5, 3.4000000000000007e-5, 0.002, 0.002, 0.002, 0.002, 3.6e-5, 3.6e-5, 3.6e-5, 0.002, 0.002, 0.00

PauliSum(nqubits: 4, 28 Pauli terms:
 -0.65018 * IIZY
 -0.27738 * IYZI
 0.44595 * IIXI
 -0.22553 * IIYZ
 -0.36244 * ZXZI
 1.1821 * IZZI
 -0.50312 * IZXZ
 1.0528 * XIII
 -0.08529 * IIYY
 -0.23527 * ZYII
 -0.24015 * IZYI
 -0.10442 * ZXXZ
 0.06221 * YZII
 -0.11051 * YXXZ
 0.51771 * IIIX
 -0.043696 * ZXYI
 -0.08974 * IYXZ
 -0.031359 * ZXXY
 -0.20093 * IZXY
 -0.028358 * IYYI
  ⋮)

In [17]:
# overlap with the initial state should give ground state of the target Hamiltonian (TFIM)
overlapwithplus(propagated)

2.444716527225737

In [ ]:
# check hyperparameter tuning
# are signs of j and gamma correct?
# can we get accurate gs without using adapt?